<a href="https://colab.research.google.com/github/Kushal94gitgit/Research-project/blob/main/inversecode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import joblib, json

fwd_model = keras.models.load_model(
    "bandgap_mlp_keras.h5",
    compile=False
)

scaler_X = joblib.load("scaler_X.pkl")

with open("feature_names.json", "r") as f:
    feature_names = json.load(f)

n_features = len(feature_names)


In [18]:
fwd_model.trainable = False


In [19]:
def build_inverse_nn(n_features):
    inp = keras.Input(shape=(1,))
    x = layers.Dense(64, activation="relu")(inp)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dense(256, activation="relu")(x)
    out = layers.Dense(n_features, activation="linear")(x)
    return keras.Model(inp, out)

inv_model = build_inverse_nn(n_features)


In [20]:
target_bg = keras.Input(shape=(1,))

# wrap inverse model
x_scaled = keras.layers.Lambda(
    lambda x: inv_model(x),
    name="inv_wrapper"
)(target_bg)

# project 17 -> 24
x_scaled = keras.layers.Dense(24, name="projection_17_to_24")(x_scaled)

# wrap forward model
bg_pred = keras.layers.Lambda(
    lambda x: fwd_model(x),
    name="fwd_wrapper"
)(x_scaled)

inverse_pipeline = keras.Model(
    inputs=target_bg,
    outputs=bg_pred,
    name="inverse_pipeline"
)

inverse_pipeline.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="mse"
)


In [21]:
target_bandgaps = np.linspace(0.5, 5.5, 200).reshape(-1, 1)

inverse_pipeline.fit(
    target_bandgaps,
    target_bandgaps,
    epochs=300,
    batch_size=32,
    verbose=2
)


Epoch 1/300
7/7 - 1s - 211ms/step - loss: 7.8978
Epoch 2/300
7/7 - 0s - 10ms/step - loss: 7.3781
Epoch 3/300
7/7 - 0s - 9ms/step - loss: 7.2241
Epoch 4/300
7/7 - 0s - 9ms/step - loss: 6.7474
Epoch 5/300
7/7 - 0s - 10ms/step - loss: 6.4038
Epoch 6/300
7/7 - 0s - 10ms/step - loss: 5.9736
Epoch 7/300
7/7 - 0s - 10ms/step - loss: 5.3428
Epoch 8/300
7/7 - 0s - 10ms/step - loss: 4.6475
Epoch 9/300
7/7 - 0s - 12ms/step - loss: 4.1268
Epoch 10/300
7/7 - 0s - 10ms/step - loss: 3.4578
Epoch 11/300
7/7 - 0s - 10ms/step - loss: 3.0126
Epoch 12/300
7/7 - 0s - 10ms/step - loss: 2.5016
Epoch 13/300
7/7 - 0s - 11ms/step - loss: 2.0303
Epoch 14/300
7/7 - 0s - 10ms/step - loss: 1.6002
Epoch 15/300
7/7 - 0s - 10ms/step - loss: 1.1887
Epoch 16/300
7/7 - 0s - 10ms/step - loss: 0.8527
Epoch 17/300
7/7 - 0s - 10ms/step - loss: 0.5166
Epoch 18/300
7/7 - 0s - 10ms/step - loss: 0.3564
Epoch 19/300
7/7 - 0s - 10ms/step - loss: 0.2516
Epoch 20/300
7/7 - 0s - 9ms/step - loss: 0.2179
Epoch 21/300
7/7 - 0s - 9ms/ste

In [22]:
# define once
projection = keras.layers.Dense(24, name="projection_17_to_24")

# use everywhere
x_scaled_24 = projection(x_scaled_17)


In [23]:
bg_request = np.array([[2.0]])  # desired bandgap in eV

# inverse model output (17)
x_scaled_17 = inv_model.predict(bg_request)

# project 17 -> 24 (use the SAME Dense layer weights if trained, or define once)
projection = keras.layers.Dense(24)
x_scaled_24 = projection(x_scaled_17)

# now inverse scale
x_pred = scaler_X.inverse_transform(x_scaled_24)

for name, val in zip(feature_names, x_pred[0]):
    print(name, val)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
Sr.No. 23.381781563368676
material_id 21.520059683383753
formula_pretty 2.5492395538291888
band_gap 4.737369564772865
energy_above_hull 1.5484470022946264
formation_energy_per_atom 0.046525418265001325
predicted_formation_energy_per_atom -1.5960704769895686
density 0.06739589813567584
volume 7.681228407710168
nsites 83.43569238639026
Elements 8.17402019076638
lattice_a 82.61554353265511
lattice_b 10.876238244872642
lattice_c 85.34192396627346
lattice_alpha 48.35061367137631
lattice_beta 48.3460502371783
lattice_gamma 9.270580276275194
